# ERA5 comparison with CMIP6 models

In this tutorial we will compare the ERA5 reanalysis to CMIP6 model outputs. To do so, you will choose a model from your country for example.

Getting CMIP6 data can be a bit painful via the official [ESGF portal](https://esgf-node.llnl.gov/search/cmip6/). The easiest way is usually to use the data directly from a computing center. In this tutorial, we will use the [instake-esm](https://intake-esm.readthedocs.io/en/stable/) package that allow to directly load CMIP6 data with Python: https://intake-esm.readthedocs.io/en/stable/user-guide/cmip6-tutorial.html

This is made possible thanks to partnership with Google Cloud: [New climate model data now in Google Public Datasets](https://cloud.google.com/blog/products/data-analytics/new-climate-model-data-now-google-public-datasets). This is rather designed to work on [Pangeo-Cloud](https://pangeo.io/cloud.html) where the servers are directly connected to the data. However we can access it from anywhere with less performance. Note that intake can also be installed in your data center if data are already there.

## Import packages

In [ ]:
# To reload external files automatically (ex: utils)
%load_ext autoreload
%autoreload 2

import xarray as xr
import dask
import intake
import pprint
import pandas as pd
import numpy as np
import calendar as cld
import matplotlib.pyplot as plt
import proplot as plot # New plot library (https://proplot.readthedocs.io/en/latest/)
plot.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures
from scipy import stats
import xesmf as xe # For regridding (https://xesmf.readthedocs.io/en/latest/)

# Import some extra functions from utils folder
import sys
sys.path.insert(1, 'utils') # to include the util directory
import utils as u # my personal functions
u.check_python_version()
u.check_virtual_memory()

## Loading a catalog

In [ ]:
url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
col

## Catalog Contents

In [ ]:
col.df.head()

## Finding unique entries

In [ ]:
uni_dict = col.unique(["source_id", "experiment_id", "table_id"])
pprint.pprint(uni_dict, compact=True)

## Searching for specific datasets

In the example below, we are are going to search for the following:
- variables: `tas` which stands for near-surface (usually, 2 meter) air temperature ([IPCC Standard Output from Coupled Ocean-Atmosphere GCMs](https://pcmdi.llnl.gov/mips/cmip3/variableList.html))

- experiments: `['historical', 'sspxxx']`:
    - `historical`: all forcing of the recent past.
    - `sspxxx`: emission-driven Shared Socioeconomic Pathways ([O'Neill et al, 2016](https://doi.org/10.5194/gmd-9-3461-2016)).


- table_id: `Amon` which stands for monthly mean variables on the atmosphere grid.

- source_id: `IPSL-CM6A-LR` which stands for a model (choose one from your country for example).
    - search your country from here: [CMIP6_institution_id](https://wcrp-cmip.github.io/CMIP6_CVs/docs/CMIP6_institution_id.html)
    - find one model: [CMIP6_source_id](https://wcrp-cmip.github.io/CMIP6_CVs/docs/CMIP6_source_id.html) (check that `ScenarioMIP` is available)
    

- member_id: `r1i1p1f1` to get the first member.

For more details on the CMIP6 vocabulary, please check this [website](http://clipc-services.ceda.ac.uk/dreq/index.html), and [Core Controlled Vocabularies (CVs) for use in CMIP6](https://github.com/WCRP-CMIP/CMIP6_CVs) GitHub repository.

In [ ]:
model_name = ### choose one model from your country

cat = col.search(
    experiment_id=["historical", "ssp119", "ssp126", "ssp245", "ssp370", "ssp585"],
    table_id="Amon",
    variable_id="tas",
    source_id=model_name,
    member_id="r1i1p1f1"
)

cat

In [ ]:
model_name = 'IPSL-CM6A-LR'

cat = col.search(
    experiment_id=["historical", "ssp119", "ssp126", "ssp245", "ssp370", "ssp585"],
    table_id="Amon",
    variable_id="tas",
    source_id=model_name,
    member_id="r1i1p1f1"
)

cat

In [ ]:
cat.df.head()

## Loading datasets

In [ ]:
dset_dict = cat.to_dataset_dict(
    zarr_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True}
)

In [ ]:
[key for key in dset_dict.keys()]

In [ ]:
ds = dset_dict[###] # Take the historical key of the model you have chosen
ds

In [ ]:
ds = dset_dict["CMIP.IPSL.IPSL-CM6A-LR.historical.Amon.gr"]
ds

## Visualize data

In [ ]:
model = ds.tas.isel(member_id=0) - 273.15
model

### Exercise: Compute climatology and plot it

In [ ]:
clim_model = ###

### Solution

In [ ]:
clim_model = model.mean('time').load()

In [ ]:
cmap='RdBu_r'
levels=plot.arange(-30,30,5)
extend='both'

fig, axs = plot.subplots(nrows=1, ncols=1, proj='cyl', axwidth=5)

axs[0].contourf(
    clim_model, colorbar='r', cmap=cmap, levels=levels, extend=extend, 
    colorbar_kw={'label': 'Near-surface air temperature [°C]'}, globe=True
)

year_start = str(model['time.year'].min().values.item(0))
year_end = str(model['time.year'].max().values.item(0))

axs.format(
    labels=True, coast=True, borders=True,
    suptitle=model_name+' near-surface air temperature climatology ('+year_start+'-'+year_end+')'
)

## Compare ERA5 to model
See back `02_ERA5.ipynb`

### Download ERA5

1. Go to https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5
2. Search `ERA5 monthly averaged data on single levels from 1979 to present`
3. Download:
    - Product type: Monthly averaged reanalysis
    - Variable: 2m temperature
    - Select all years (except 2021) / months / time
    - Geographical area: Whole available region
    - Format: NetCDF (experimental)
4. Login/register to submit request (create an account if you don't have one)
5. Go back down on the page and click on Submit Form
6. Click on download, cancel, then right click on the download button and copy the link path, then paste it on the cell bellow besides the `wget` command

Remark: It should be around 1Go

In [ ]:
!wget https://download-0008.copernicus-climate.eu/cache-compute-0008/cache/data6/adaptor.mars.internal-1642500706.3691368-19649-5-5a435782-6700-4aa6-991b-f43bad55b9eb.nc

7. Rename the downloaded file to `ERA5.nc`

In [ ]:
ds = xr.open_dataset('ERA5.nc', chunks={"longitude": 360, "latitude": 360}) \
        .rename({'longitude': 'lon', 'latitude': 'lat'})
obs = ds.t2m - 273.15

## Select same time period
Before making the comparison, be aware that the two data sets are not from the same time period.

### Exercise
Check the period of the model data and ERA5 to choose a common period

In [ ]:
clim_obs = ###
clim_model = ###

### Solution

In [ ]:
# - model: 1979-2014
# - obs: 1850-2020

period = slice('1979', '2014')

# Test that both data sets are having the same size
np.testing.assert_equal(
    obs.sel(time=period).time.size,
    model.sel(time=period).time.size
)

clim_obs = obs.sel(time=period).mean('time').load()
clim_model = model.sel(time=period).mean('time').load()

## Regrid ERA5 to the model grid
We cannot directly compare ERA5 to the model because they are not on the same grid. For that we will have to make a regrid. Several options:
- use command line tools like [CDO](https://code.mpimet.mpg.de/projects/cdo/)
- use [xESMF](https://xesmf.readthedocs.io/en/latest/) which is an adaptation of a Fortran program in Python and takes into account the sphericity of the Earth (I recommend, but it only works on Linux and maybe Mac)
- for non-Linux users, you can also directly use the [`.interp()`](https://xarray.pydata.org/en/stable/user-guide/interpolation.html#example) function of xarray, which does not take into account the sphericity of the Earth but does the job.

### For Linux users (xESMF)
Check: [Regrid between rectilinear grids](https://xesmf.readthedocs.io/en/latest/notebooks/Rectilinear_grid.html)

In [ ]:
# Create a regridder
regridder = xe.Regridder(
    clim_obs, clim_model, 'bilinear', periodic=True, reuse_weights=True
)

In [ ]:
# Make the regrid
clim_obs_regrid = regridder(clim_obs)
clim_obs_regrid

### For non-Linux users (xarray)
Check: https://xarray.pydata.org/en/stable/user-guide/interpolation.html#example

In [ ]:
clim_obs_regrid = clim_obs.interp(lat=clim_model.lat, lon=clim_model.lon)
clim_obs_regrid

### Plot

In [ ]:
cmap='RdBu_r'
levels=plot.arange(-4,20,2)
extend='both'
norm='div'

# Make a zoom on your country
latmin=38 ; latmax=56 ; lonmin=-10 ; lonmax=15

fig, axs = plot.subplots(nrows=1, ncols=3, proj='cyl', axwidth=3)
  
m = axs[0].pcolormesh(clim_model, cmap=cmap, levels=levels, extend=extend, norm=norm)
axs[0].format(title=model_name) 

axs[1].pcolormesh(clim_obs, cmap=cmap, levels=levels, extend=extend, norm=norm)
axs[1].format(title='ERA5') 

axs[2].pcolormesh(clim_obs_regrid, cmap=cmap, levels=levels, extend=extend, norm=norm)
axs[2].format(title='ERA5 (regrid)')

fig.colorbar(m, label='Near-surface air temperature [°C]')

axs.format(
    labels=True, coast=True, borders=True,
    suptitle='Near-surface air temperature climatology ('+period.start+'-'+period.stop+')',
    latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
)

## Plot the global bias
Or take a sub-zone if it's too heavy

### Exercise
Make a figure showing the annual climatological bias in surface temperature between the model and ERA5 (model - ERA5).

### Solution

In [ ]:
cmap='RdBu_r'
levels=plot.arange(-30,30,5)
extend='both'

fig, axs = plot.subplots(nrows=3, ncols=1, proj='cyl', axwidth=5)

# Obs
m1 = axs[0].contourf(
    clim_obs_regrid, cmap=cmap, levels=levels, extend=extend, globe=True
)
axs[0].format(title='ERA5 (regrid)')

# Model
axs[1].contourf(
    clim_model, cmap=cmap, levels=levels, extend=extend, globe=True
)
axs[1].format(title=model_name)

# Bias (model - obs)
m2 = axs[2].contourf(
    clim_model-clim_obs_regrid, cmap=cmap, 
    levels=plot.arange(-5, 5, 1), extend=extend, globe=True
)
axs[2].format(title='Bias ('+model_name+' - ERA5)')

fig.colorbar(m1, rows=(1, 2), label='Near-surface air temperature [°C]')
fig.colorbar(m2, row=3, label='Bias [°C]')

axs.format(
    labels=True, coast=True, borders=True,
    suptitle='Near-surface air temperature climatologies and bias ('+period.start+'-'+period.stop+')'
)

In [ ]:
cmap='RdBu_r'
levels=plot.arange(-30,30,5)
extend='both'

fig, axs = plot.subplots(nrows=1, ncols=1, proj='cyl', axwidth=5)

# Bias (model - obs)
m = axs[0].contourf(
    clim_model-clim_obs_regrid, cmap=cmap, 
    levels=plot.arange(-5, 5, 1), extend=extend, globe=True
)

fig.colorbar(m, label='Temperature bias [°C]')

axs.format(
    labels=True, coast=True, borders=True,
    suptitle='Near-surface air temperature bias '+model_name+' - ERA5 ('+period.start+'-'+period.stop+')'
)
# fig.save('img/'+model_name+'_tas_bias_'+period.start+'-'+period.stop+'.jpg')

## Plot global time series
If too heavy take a sub-zone

In [ ]:
# Resample by year
obs_year = obs.resample(time='Y').mean('time').load()
model_year = model.resample(time='Y').mean('time').load()

In [ ]:
# Make spatial average
ts_obs = (u.spatial_average(obs_year)).load()
ts_model = (u.spatial_average(model_year)).load()

In [ ]:
fig, axs = plot.subplots(axwidth=5, aspect=2)

axs[0].plot(ts_obs['time.year'], ts_obs, label='ERA5')
axs[0].plot(ts_model['time.year'], ts_model, label=model_name)

    
axs[0].legend()

axs.format(
    xlabel='year',
    ylabel='Near-surface air temperature [°C]',
    suptitle='Global near-surface air temperature',
)

## Add projections
Take anomalies with respect to a common time period, e.g. 1995-2014, to get rid of the global bias.

Check this for using IPCC Color Palettes: https://pyam-iamc.readthedocs.io/en/stable/tutorials/ipcc_colors.html (this package is not installed here)

Get back the keys to load scenarios data sets!

In [ ]:
[key for key in dset_dict.keys()]

### Exercise
Make a figure showing the near-surface air temperature anomalies over the historical period and the future period up to 2100 for all scenarios available for your model.

### Solution

In [ ]:
list_scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
colors = ['#00a9cf', '#003466', '#f69320', '#df0000', '#980002']

In [ ]:
temp = []
for scenario in list_scenarios:
    ds = dset_dict["ScenarioMIP.IPSL.IPSL-CM6A-LR."+scenario+".Amon.gr"]
    da = ds.tas.isel(member_id=0) - 273.15
    temp.append(u.spatial_average(da.resample(time='Y').mean('time')))

# Concatenate results
ts_model_future = xr.concat(temp, pd.Index(list_scenarios, name='scenario')).load()

In [ ]:
ref_period = slice('1995', '2014')
ref_obs = ts_obs.sel(time=ref_period).mean('time')
ref_model = ts_model.sel(time=ref_period).mean('time')

linewidth = 1

fig, axs = plot.subplots(axwidth=5, aspect=2)

# Past
axs[0].plot(
    ts_obs['time.year'], 
    ts_obs-ref_obs, 
    label='ERA5', color='k', linewidth=linewidth, linestyle='--'
)
axs[0].plot(
    ts_model['time.year'], 
    ts_model-ref_model, 
    label='historical', color='k', linewidth=linewidth
)

# Future
for i, scenario in enumerate(list_scenarios):
    axs[0].plot(
        ts_model_future['time.year'], 
        ts_model_future.sel(scenario=scenario) - ref_model, 
        label=scenario, color=colors[i], linewidth=linewidth
    )
    
axs[0].legend(ncols=2)

axs.format(
    xlabel='year',
    ylabel='Near-surface air temperature [°C]',
    suptitle='Global near-surface air temperature anomalies '+model_name+'\n(with respect to 1995-2014)',
    xlim=(1979,2100)
)
# fig.save('img/'+model_name+'_ts_tas_anomalies_projections.jpg')

## Share your results!
Go to the following link and upload the figures for your the model you chose of the global (or sub-zone) bias and projections!

https://app.mural.co/t/variabiliteclimatique4363/m/variabiliteclimatique4363/1639491102483/fecf896fc166a77732b34ff63bf09000883718e3?sender=ufcbfba826e94d93c633c7410

## Bonus
If you still have time, try adding models or members to add uncertainties (to add envelopes to the curves, you can look at the [`fill_between()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.fill_between.html) function of matplotlib or otherwise proplot implements it directly in a simplified way: https://proplot.readthedocs.io/en/v0.6.4/1dplots.html#Shading-and-error-bars). 

Feel free to do any other analysis you are interested in!